In [ ]:
import requests  # Imports the 'requests' library, used for making HTTP requests.
import json      # Imports the 'json' library, used for working with JSON data.

In [ ]:
def create_payload(model, messages, stream=False):
    """
    Creates the payload for the API request.

    Args:
        model (str): The model to use for the API request.
        messages (list): A list of message dictionaries, where each message
                         contains 'role' and 'content' keys.
        stream (bool, optional): Whether to stream the response. Defaults to False.

    Returns:
        dict: The payload dictionary to be sent in the API request.
    """
    return {
        "model": model,
        "messages": messages,
        "stream": stream
    }

In [ ]:
def make_api_request(url, payload, headers, stream=False):
    """
    Makes a POST request to the specified URL with the given payload and headers.

    Args:
        url (str): The URL to send the request to.
        payload (dict): The data to send in the request body.
        headers (dict): The headers to include in the request.
        stream (bool, optional): Whether to stream the response. Defaults to False.

    Returns:
        requests.Response: The response object from the request, or None if the request failed.
    """
    try:
        # Attempt to make a POST request to the specified URL.
        response = requests.post(url, data=json.dumps(payload), headers=headers, stream=stream)

        # Raise an HTTPError for bad responses (4xx or 5xx status codes).
        response.raise_for_status()

        # If the request was successful, return the response object.
        return response

    except requests.exceptions.RequestException as e:
        # Catch any exceptions that occur during the request (e.g., network errors, invalid URLs).
        # Print an error message with the exception details.
        print(f"Request failed: {e}")

        # Return None to indicate that the request failed.
        return None

In [ ]:
def process_streamed_response(response):
    """
    Processes a streamed API response and prints the content chunk by chunk.

    Args:
        response (requests.Response): The API response object.
    """
    # Check if a valid response was received.
    if response:
        # Iterate through each line (chunk) of the streamed response.
        for chunk in response.iter_lines(decode_unicode=True):
            # Check if the chunk is not empty.
            if chunk:
                try:
                    # Attempt to parse the chunk as JSON.
                    data = json.loads(chunk)
                    # Extract the 'content' from the parsed JSON and print it.
                    # end='' prevents newline characters from being printed, allowing content to be streamed on one line.
                    # flush=True ensures that the output is immediately printed to the console.
                    print(data.get('message', {}).get('content', ''), end='', flush=True)
                except json.JSONDecodeError as e:
                    # Handle JSON decoding errors by printing an error message.
                    print(f"JSON decoding error: {e}")
    else:
        # If no valid response was received, print an API request failure message.
        print("API request failed.")

In [ ]:
def process_non_streamed_response(response):
    """
    Processes a non-streamed API response and prints the extracted content.

    Args:
        response (requests.Response): The API response object.
    """
    # Check if a valid response was received from the API.
    if response:
        # If a valid response was received, extract the 'content' from the response.
        content = extract_content(response)
        # Print the extracted content to the console.
        print(content)
    else:
        # If no valid response was received, print an error message indicating API request failure.
        print("API request failed.")

In [ ]:
def extract_content(response):
    """
    Extracts the 'content' field from the API response JSON.

    Args:
        response (requests.Response): The API response object.

    Returns:
        str: The extracted content, or a default message if not found.
    """
    try:
        # Attempt to parse the API response as JSON.
        result = response.json()
        # Extract the 'content' field from the parsed JSON.
        # Use .get() with default values to handle cases where 'message' or 'content' keys are missing.
        return result.get('message', {}).get('content', 'No content found')
    except (json.JSONDecodeError, AttributeError):
        # Handle exceptions that may occur during JSON parsing or attribute access.
        # Return an error message indicating that parsing failed or content was not found.
        return "Error parsing response or content not found."

In [ ]:
# Define the API endpoint URL.
url = "http://localhost:11434/api/chat"

# Specify the model to be used for the chat.
model = "llama3.2"

# Define the messages to send to the API. In this case, a single user message.
messages = [
  {
    "role": "user", 
    "content": "Explain Mixture of Experts"
  }
]

# Set the streaming flag. If True, the response will be streamed; otherwise, it will be returned as a single JSON object.
stream = True 

# Create the payload for the API request using the create_payload function.
payload = create_payload(model, messages, stream)

# Define the headers for the API request, specifying that the content type is JSON.
headers = {"Content-Type": "application/json"}

# Make the API request using the make_api_request function. The stream flag is passed to handle streaming responses.
response = make_api_request(url, payload, headers, stream=stream)

# Check the value of the stream flag to determine how to process the response.
if stream:
    # If streaming is enabled, process the response using the process_streamed_response function.
    process_streamed_response(response)
else:
    # If streaming is disabled, process the response using the process_non_streamed_response function.
    process_non_streamed_response(response)